---

# Code des Flags 1 à 4



---

# Flag1 
register

On se rend dans le datacenter. Sur l'inscription il est écrit : POST-IT PASSWORD: ISECR0XX. Sur le post-it en dessous du clavier il y'a marqué : "U2FsdGVkX1/ayza2lqhPr8Qsxi3DLMV+/rtq/ywiXzsJFWOZupP7MiKlk0QJF/CG
WEDOjPZGcXQfLLwtETC9iA==" 
Le but du flag est de déchiffrer le chiffré du post it à l'aide du password ISECR0XX. Ensuite, on peut créer un compte sur le terminal du datacenter


In [4]:
import subprocess

#passphrase =ISECR0XX
#cryptext = U2FsdGVkX1/ayza2lqhPr8Qsxi3DLMV+/rtq/ywiXzsJFWOZupP7MiKlk0QJF/CGWEDOjPZGcXQfLLwtETC9iA==
def decrypt(passphrase,cipher='aes-128-cbc'):
    # prépare les arguments à envoyer à openssl
    pass_arg = 'pass:{}'.format(passphrase)
    args = ['openssl', 'enc','-d','-base64','-' + cipher, '-pbkdf2', '-pass', pass_arg]
    fichier = open("ciphertext.txt", "r")
    chiffre=fichier.read()

    # si plaintext est de stype str, on est obligé de l'encoder en bytes pour
    # pouvoir l'envoyer dans le pipeline vers openssl
    if isinstance(chiffre, str):
        chiffre=chiffre.encode('utf-8')

    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=chiffre, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        print (error_message)
    
    fichier.close()

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout.decode()

passphrase ="ISECR0XX"

print("message déchiffré : ",decrypt(passphrase,cipher='aes-128-cbc'))



message déchiffré :  386b8f72-57d8-47c0-af4c-5c7d3f795f39


# FLAG2
pki.tutorial

On va dans le local de service 3 et on suit les étapes du tutoriel de l'option Public Key Infrastructure : tutoriel

Le but du flag est de chiffré le message "I got it!" à l'aide de la clé publique de l'utilisateur "pki_tutorial".

In [5]:
def encrypt_pkey():
    # prépare les arguments à envoyer à openssl
    args = ['openssl', 'pkeyutl', '-encrypt', '-pubin', '-inkey', 'keys/pkey_tuto.pem']

    fichier=open('pki_tutorial.txt','r')
    message=fichier.read()
    # si plaintext est de stype str, on est obligé de l'encoder en bytes pour
    # pouvoir l'envoyer dans le pipeline vers openssl
    if isinstance(message, str):
        message=message.encode('utf-8')

    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=message, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        print (error_message)
    
    fichier.close()

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout.hex()

print("chiffré :",encrypt_pkey())

chiffré : 16a9e8f3be7bba3fd80f6b68792a7488f7b36bcdffb17869c2583d1711a485ec896b342b531377fe60cdbc84444fed6f605fa1b08038344eaf5cb2c4822872229753fb31334cb8b7f750e6a62c2dbfe78057f276a94d2dfd365e0319c37be424619464dc8618fa0eff134f040928a90504b178c865275e484a7ae78313cfead3b03b2c975a95cd6feda03123779bde10402b97e7e900f8130f7da7f0c483d13d29dc239d106975fa6500caaa9b00829d867e2a3612a6f932457448eb81bfa83f154828471cc8ccbace98cecd5f7508e254df0319e91ec77b93c1ab72118dd95e38eae4354f90e353b8a46d7a3d48bd241cf256145dd1de7cd80eb5bdf79fe190


# FLAG3
pki.upload

On génère une paire de clé de 2048 bits avec :
$openssl genpkey -algorithm RSA -pkeyopt rsa_keygen_bits:2048

On stocke le résultat dans un fichier "keys/priv_key.pem"

On extrait la clé publique à l'aide de :
$openssl pkey -in keys/priv_key.pem -pubout 

On stocke le résultat dans un autre fichier "keys/pub_key.pem"

Pour avoir le flag il faut copier coller cette clé publique dans un terminal de service à l'option Public Key Infrastructure : Upload public-key

## FLAG4
power.on

Aller dans le local électrique et taper 1. On nous donne un message à signer avec la clé privée qu'on a généré pour le flag3.
On signe alors le message "kebob enrol lotto chips hades"

In [6]:
#on donne en argument à sign le message à signer et le chemin de la clé privée à utiliser pour signer.
def sign(plain_text,chemin_priv_key):
    # prépare les arguments à envoyer à openssl
    args = ['openssl', 'dgst', '-sha256', '-sign', chemin_priv_key,'-hex']

    # fichier=open('sign_elec.txt','r')
    message=plain_text
    # si plaintext est de stype str, on est obligé de l'encoder en bytes pour
    # pouvoir l'envoyer dans le pipeline vers openssl
    if isinstance(message, str):
        message=message.encode('utf-8')

    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=message, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        print (error_message)
    

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout



print("signature :", sign("kebob enrol lotto chips hades","keys/priv_key.pem"))



signature : b'SHA2-256(stdin)= 9d043c68a3a9058ea6db3183df9667886d13101e58a11ff965becb54e4373bbd2290cc0a2639d0847f14acbb1fba5badae17f315dea25f6ca6465e099a829fc0f957fca3e028e9daea2a497572155dbb223bc0efcf7c1a0a2757989f4b2d6b05771c32ff00426452ec3dc3d40475b5036e2e96fa835e006696facf6d9b0714bb4552fc672bec50a7443391b4b5ff400b90beec9b5b0202680d4d81bf22138a35f36fade864b686abe2446c73cfd90d41e136eb78131ed68fb86ed54c8879779afa71d949ad7caa11c0fbf2588f520f76f482c9c55a524bf42cf82b1b69bdad4de7c6d063d5d13e192a1328697e4c94ef69bb4f2f15938a9219844178d73aa511\n'
